In [ ]:
print("This is the test for initial commit!")

In [ ]:
import pandas as pd
print("Pandas imported successfully, version: "+pd.__version__)
import statsmodels as sm
print("Statsmodels imported successfully, version: "+sm.__version__)
import plotly.graph_objects as go
print("Plotly function imported succesfully")
import plotly.express as px
print("Plotly express imported succesfully")
from plotly.subplots import make_subplots

import plotly as py

import plotly.figure_factory as ff

import matplotlib.pyplot as plt

import numpy as np

from statsmodels.graphics.gofplots import qqplot
print("Succesful")

In [ ]:
data_all = pd.read_csv('/Users/mazutislab/Desktop/SynBio/US_Accidents_May19.csv')

In [ ]:
pd.set_option('display.max_columns', 50)
data_all.head()

In [ ]:
#Selecting only needed data in the given dataset
data_cleaned = data_all[["Severity", "Temperature(F)", "Humidity(%)", "Pressure(in)", "Visibility(mi)", "Wind_Speed(mph)", 
                        "Precipitation(in)"]]
data_cleaned.reset_index()
type(data_cleaned)

#Convert Temperature in Fahrenheit to Celsius
def fahr_to_celsius(temp_fahr):
    """Convert Fahrenheit to Celsius and Return Celsius conversion of input"""
    temp_celsius = (temp_fahr - 32) * 5 / 9
    return temp_celsius

data_cleaned["Temperature_C"] = (fahr_to_celsius(data_cleaned["Temperature(F)"])).round(2)
data_cleaned.drop(['Temperature(F)'], inplace = True, axis = 1)

#Convert Pressure in inches of mercury to mbar
data_cleaned["Pressure(mbar)"] = data_cleaned["Pressure(in)"]*0.033863886666667*1000
data_cleaned.drop(['Pressure(in)'], inplace = True, axis = 1)

#Convert Wind Speed in mph to kmh
data_cleaned["Wind_Speed(kmh)"] = data_cleaned["Wind_Speed(mph)"]*1.609344
data_cleaned.drop(['Wind_Speed(mph)'], inplace = True, axis = 1)

#Convert Precipitation in inches to mm
data_cleaned["Precipitation(mm)"] = data_cleaned["Precipitation(in)"]*25.4
data_cleaned.drop(['Precipitation(in)'], inplace = True, axis = 1)

#Convert Visibility in miles to km
data_cleaned["Visibility(km)"] = data_cleaned["Visibility(mi)"]*1.609344
data_cleaned.drop(['Visibility(mi)'], inplace = True, axis = 1)
data_cleaned.head()

In [ ]:
data_cleaned.describe()

In [ ]:
fig = go.Figure(data=[go.Histogram(x=(data_cleaned['Severity']))])

fig.show()

#Frequency of severities
Severity_frequency = {}

for i, u in enumerate(range(0,5)):
    Severity_frequency['Severity' + str(u)] = (data_cleaned[(data_cleaned['Severity']==i)]).shape[0]

print(Severity_frequency)


In [ ]:
data_cleaned = data_cleaned[data_cleaned['Severity'] >= 1]
data_cleaned = data_cleaned.sort_values(by=['Severity'], ascending = True)
data_cleaned.reset_index()
data_cleaned.head()

In [ ]:
#Histogram Temperature
hist_temperature = go.Figure(data=[go.Histogram(x=(data_cleaned['Temperature_C']))])

hist_temperature.update_layout(
    xaxis_title="Temperature (\u00b0C)",
    yaxis_title="Frequency"
)

hist_temperature.show()

#Lowest/Highest Temp
print("Lowest temperature in dataset is " + str((data_cleaned['Temperature_C']).min()) + " \u00b0C")

print("Highest temperature in dataset is " + str((data_cleaned['Temperature_C']).max()) + " \u00b0C")

#Box Plot Temperature
box_plot_temperature = px.box(data_cleaned, x="Severity", y="Temperature_C")
box_plot_temperature.show()

#Outlier Analysis

temperature_values_low = data_cleaned['Temperature_C'].between(-62, -30, inclusive = True)
print("There are " + str((data_cleaned[temperature_values_low]).shape[0]) + " temperature values between -62 and -30 \u00b0C")

temperature_values_high = data_cleaned['Temperature_C'].between(45, 80, inclusive = True)
print("There are " + str((data_cleaned[temperature_values_high]).shape[0]) + " temperature values between 45 and 80 \u00b0C")




In [ ]:
#Temperature Histograms

#Severity 1

sev1_temp = ((data_cleaned[(data_cleaned['Severity']) == 1])['Temperature_C'])
sev2_temp = ((data_cleaned[(data_cleaned['Severity']) == 2])['Temperature_C'])
sev3_temp = ((data_cleaned[(data_cleaned['Severity']) == 3])['Temperature_C'])
sev4_temp = ((data_cleaned[(data_cleaned['Severity']) == 4])['Temperature_C'])


fig = make_subplots(rows=2, cols=2, subplot_titles=("Severity 1","Severity 2", "Severity 3", "Severity 4"))

#Adding Traces
fig.add_trace(go.Histogram(x=(sev1_temp), nbinsx=20), row=1, col=1)
fig.add_trace(go.Histogram(x=(sev2_temp), nbinsx=20), row=1, col=2)
fig.add_trace(go.Histogram(x=(sev3_temp), nbinsx=20), row=2, col=1)
fig.add_trace(go.Histogram(x=(sev4_temp), nbinsx=20), row=2, col=2)

# Update xaxis properties
fig.update_xaxes(title_text="Temperature \u00b0C", row=1, col=1)
fig.update_xaxes(title_text="Temperature \u00b0C", row=1, col=2)
fig.update_xaxes(title_text="Temperature \u00b0C", row=2, col=1)
fig.update_xaxes(title_text="Temperature \u00b0C", row=2, col=2)

# Update yaxis properties
fig.update_yaxes(title_text="Frequency", row=1, col=1)
fig.update_yaxes(title_text="Frequency", row=1, col=2)
fig.update_yaxes(title_text="Frequency", row=2, col=1)
fig.update_yaxes(title_text="Frequency", row=2, col=2)

# Update title and height
fig.update_layout(title_text="Variable Temperature_C histograms for each severity group", height=800)

fig.show()

In [ ]:
from scipy.stats import normaltest
sev1_temp = ((data_cleaned[(data_cleaned['Severity']) == 1])['Temperature_C'])
sev1_temp = sev1_temp.to_numpy()
sev1_temp = sev1_temp[np.logical_not(np.isnan(sev1_temp))]
#sev1_temp = np.random.choice(sev1_temp, 50000)

from scipy.stats import anderson

#1
result = anderson(sev1_temp)
stat = round(result.statistic, 4)

p = 0
result_mat = []
for i in range(len(result.critical_values)):
    title = "Severity " + str(int((i+3)%(i+2)))
    sl, cv = result.significance_level[i], result.critical_values[i]
    if result.statistic < result.critical_values[i]:
        msg = 'Sample looks Gaussian (fail to reject H0)'
    else:
        msg = 'Sample does not look Gaussian (reject H0)'
    result_mat.append([title, len(sev1_temp), stat, sl, cv, msg])

trace = go.Table(
    header=dict(values=['<b>Title<b>', '<b>Sample Size</b>', '<b>Statistic</b>', '<b>Significance Level</b>', '<b>Critical Value</b>', '<b>Comment</b>'],
                line = dict(width=0),
                fill = dict(color='rgba(42,63,95,0.8)'),
                align = 'center',
                font = dict(
                    color = '#ffffff',
                    size = 12
                )),
    cells=dict(values=np.array(result_mat).T,
               line = dict(width=0),
               fill = dict(color=[['#EBF0F8', '#ffffff', '#EBF0F8', '#ffffff', '#EBF0F8']]),
               align = 'center',
               height = 40),
    columnwidth=[0.3, 0.25, 0.3, 0.25, 0.5])
layout = dict(
    height=300,
    margin=dict(
        l=5,
        r=5,
        t=30,
        b=0
    )
)
data = [trace]
andar_table = dict(data=data, layout=layout)

iplot(andar_table, filename='anderson-darling-table')

#2

sev2_temp = ((data_cleaned[(data_cleaned['Severity']) == 2])['Temperature_C'])
sev2_temp = sev2_temp.to_numpy()
sev2_temp = sev2_temp[np.logical_not(np.isnan(sev2_temp))]

result = anderson(sev2_temp)
stat = round(result.statistic, 4)

p = 0
result_mat = []
for i in range(len(result.critical_values)):
    title = "Severity " + str(int((i+6)%(i+4)))
    sl, cv = result.significance_level[i], result.critical_values[i]
    if result.statistic < result.critical_values[i]:
        msg = 'Sample looks Gaussian (fail to reject H0)'
    else:
        msg = 'Sample does not look Gaussian (reject H0)'
    result_mat.append([title, len(sev2_temp), stat, sl, cv, msg])

trace = go.Table(
    header=dict(values=['<b>Title<b>', '<b>Sample Size</b>', '<b>Statistic</b>', '<b>Significance Level</b>', '<b>Critical Value</b>', '<b>Comment</b>'],
                line = dict(width=0),
                fill = dict(color='rgba(42,63,95,0.8)'),
                align = 'center',
                font = dict(
                    color = '#ffffff',
                    size = 12
                )),
    cells=dict(values=np.array(result_mat).T,
               line = dict(width=0),
               fill = dict(color=[['#EBF0F8', '#ffffff', '#EBF0F8', '#ffffff', '#EBF0F8']]),
               align = 'center',
               height = 40),
    columnwidth=[0.3, 0.25, 0.3, 0.25, 0.5])
layout = dict(
    height=300,
    margin=dict(
        l=5,
        r=5,
        t=30,
        b=0
    )
)
data = [trace]
andar_table = dict(data=data, layout=layout)

iplot(andar_table, filename='anderson-darling-table')

#3

sev3_temp = ((data_cleaned[(data_cleaned['Severity']) == 3])['Temperature_C'])
sev3_temp = sev3_temp.to_numpy()
sev3_temp = sev3_temp[np.logical_not(np.isnan(sev3_temp))]

result = anderson(sev3_temp)
stat = round(result.statistic, 4)

p = 0
result_mat = []
for i in range(len(result.critical_values)):
    title = "Severity " + str(int((i+7)%(i+4)))
    sl, cv = result.significance_level[i], result.critical_values[i]
    if result.statistic < result.critical_values[i]:
        msg = 'Sample looks Gaussian (fail to reject H0)'
    else:
        msg = 'Sample does not look Gaussian (reject H0)'
    result_mat.append([title, len(sev3_temp), stat, sl, cv, msg])

trace = go.Table(
    header=dict(values=['<b>Title<b>', '<b>Sample Size</b>', '<b>Statistic</b>', '<b>Significance Level</b>', '<b>Critical Value</b>', '<b>Comment</b>'],
                line = dict(width=0),
                fill = dict(color='rgba(42,63,95,0.8)'),
                align = 'center',
                font = dict(
                    color = '#ffffff',
                    size = 12
                )),
    cells=dict(values=np.array(result_mat).T,
               line = dict(width=0),
               fill = dict(color=[['#EBF0F8', '#ffffff', '#EBF0F8', '#ffffff', '#EBF0F8']]),
               align = 'center',
               height = 40),
    columnwidth=[0.3, 0.25, 0.3, 0.25, 0.5])
layout = dict(
    height=300,
    margin=dict(
        l=5,
        r=5,
        t=30,
        b=0
    )
)
data = [trace]
andar_table = dict(data=data, layout=layout)

iplot(andar_table, filename='anderson-darling-table')

#4

sev4_temp = ((data_cleaned[(data_cleaned['Severity']) == 4])['Temperature_C'])
sev4_temp = sev4_temp.to_numpy()
sev4_temp = sev4_temp[np.logical_not(np.isnan(sev4_temp))]

result = anderson(sev4_temp)
stat = round(result.statistic, 4)

p = 0
result_mat = []
for i in range(len(result.critical_values)):
    title = "Severity " + str(int((i+9)%(i+5)))
    sl, cv = result.significance_level[i], result.critical_values[i]
    if result.statistic < result.critical_values[i]:
        msg = 'Sample looks Gaussian (fail to reject H0)'
    else:
        msg = 'Sample does not look Gaussian (reject H0)'
    result_mat.append([title, len(sev4_temp), stat, sl, cv, msg])

trace = go.Table(
    header=dict(values=['<b>Title<b>', '<b>Sample Size</b>', '<b>Statistic</b>', '<b>Significance Level</b>', '<b>Critical Value</b>', '<b>Comment</b>'],
                line = dict(width=0),
                fill = dict(color='rgba(42,63,95,0.8)'),
                align = 'center',
                font = dict(
                    color = '#ffffff',
                    size = 12
                )),
    cells=dict(values=np.array(result_mat).T,
               line = dict(width=0),
               fill = dict(color=[['#EBF0F8', '#ffffff', '#EBF0F8', '#ffffff', '#EBF0F8']]),
               align = 'center',
               height = 40),
    columnwidth=[0.3, 0.25, 0.3, 0.25, 0.5])
layout = dict(
    height=300,
    margin=dict(
        l=5,
        r=5,
        t=30,
        b=0
    )
)
data = [trace]
andar_table = dict(data=data, layout=layout)

iplot(andar_table, filename='anderson-darling-table')

In [ ]:
#Kruskal-Wallis Test

from scipy.stats import kruskal



stat, p = kruskal(sev1_temp, sev2_temp, sev3_temp, sev4_temp)
print('Statistics=%.3f, p=%.3f' % (stat, p))


# interpret
alpha = 0.05
if p > alpha:
    print('Same distributions (fail to reject H0)')
else:
    print('Different distributions (reject H0)')
    
    


In [ ]:
#Post-Hoc Test
from scipy.stats import mannwhitneyu
stat, p = mannwhitneyu(sev2_temp, sev4_temp)
print('Statistics=%.3f, p=%.3f' % (stat, p))
# interpret
alpha = 0.05
if p > alpha:
	print('Same distribution (fail to reject H0)')
else:
	print('Different distribution (reject H0)')

In [ ]:
from scipy.stats import pearsonr

data_cleaned_removed = data_cleaned[['Temperature_C', 'Severity']]
data_cleaned_removed = data_cleaned_removed.dropna()
data_cleaned_removed.head()

data_cleaned_removed_np = data_cleaned_removed.to_numpy()

corr, p_value = pearsonr(data_cleaned_removed_np[0], data_cleaned_removed_np[1])

print(p_value)
